# Feature Extraction

In [18]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader 

In [7]:
# Load Pre-Trained Model
model = models.resnet18(pretrained=True)
#model = model.to("cpu")
#remove final connected layer from model
model.fc = nn.Identity()
#model = torch.nn.Sequential(*list(model.children())[:-1])

C:\Users\20neu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\20neu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
# define image transformation
transform = transforms.Compose([
    transforms.Resize((352, 1216)),
    transforms.ToTensor()
    #transforms.Normalize()
])

In [19]:
# Set Directories
base_dir = 'dataset'
image_dir = os.path.join(base_dir, 'image')
depth_dir = os.path.join(base_dir, 'groundtruth_depth')
# Load Dataset
dataset = ImageFolder(base_dir, transform=transform)
dataloader = DataLoader(dataset, 32, False)

In [20]:
# Function to extract features
def extract_features(dataloader, model):
    features = []
    with torch.no_grad(): #Disables gradient calcullation
        for inputs, _ in dataloader:
            outputs = model(inputs)
            features.append(outputs.flatten(1))
    return torch.cat(features)

features = extract_features(dataloader, model)

# Reduce Dimensionality with LLE